In [3]:
#클러스터링 분석에 사용할 기본 패키지 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#warning 메시지 무시를 위한 코드
import warnings
warnings.filterwarnings('ignore')

In [4]:
total = pd.read_csv(r'C:\Users\aug19pm7\Desktop\데이터분석\final\total_10.csv', index_col=0)
total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103666 entries, naver to Google
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Number         103666 non-null  int64  
 1   Store          103666 non-null  object 
 2   Address        103666 non-null  object 
 3   Addr           103666 non-null  object 
 4   Review_score   103666 non-null  float64
 5   Review_counts  103666 non-null  int64  
 6   ID             103666 non-null  object 
 7   Date           103666 non-null  object 
 8   Review_text    103666 non-null  object 
 9   Time           103666 non-null  object 
 10  slope          103666 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 9.5+ MB


In [5]:
# 원본에서 일부만 사용하기
# tst = total.sample(n=1000, random_state=9)
tst = total.sample(n=1000)

In [ ]:
# 인덱스 재설정
tst = tst.reset_index(drop=True)
tst

In [46]:
txt = tst['Review_text'].apply(lambda x: x.replace(' ', ''))
txt

0    일요일기분내기위한+맛있기위한브런치로완벽!스테이크샌드위치는불고기가아니라진짜로스느낌의고...
1    신상이대브런치<포닐>정말맛있는브런치를즐길수있는이대브런치&이대맛집입니다.샌드위치,치아...
2    새우샌드위치랑아이스아메리카노주문했습니다커피도맛있고샌드위치도맛있어요!점심에갔는데포장하...
3    항상지나가면서유심히보던빵집&브런치집인데운좋게소금빵나오는시간에방문했어요☺️바질터키햄샌...
4    이대역치아바타샌드위치맛집🥪일정사이에시간이붕떠서시간도떼우고점심을먹기위해방문한곳역이랑가...
5    나의2024년1월1일첫먹방은그간너무너무너무먹고싶었던‘만동제과마늘바게트’!🧄🥖드디어소...
6           너무맛있게먹어서사진도못남겼어요마늘바게뜨가촉촉부들하고진한마늘의단맛이인상적이에요
7                            친절하십니다근데막꼭먹어야한다정도는아닌.것같아요
8                             마늘바게트에묻힌빵들너무많음홍국이깨찰빵도라이임
9    마늘빵달달하고촉촉하니맛있어요.카푸치노는커피맛나는생크림빵인데구름처럼슈르륵넘어갑니다.시...
Name: Review_text, dtype: object

In [32]:
# OpenKoreanText 토크나이저 사용해서 문장 쪼개기
# 한국어 텍스트를 형태소 단위로 분석
okt = Okt()
rv = tst['Review_text'][1]
print(rv)
# 형태소 단위로 분석하여 리스트로 반환
print(okt.morphs(rv))
# nouns를 통해 명사 추출하여 리스트로 반환
print(okt.nouns(rv))
# 형태소, 품사를 쌍으로 리스트로 반환
print(okt.pos(rv))

신상 이대 브런치 <포닐> 정말 맛있는 브런치를 즐길 수 있는 이대 브런치 & 이대 맛집입니다. 샌드위치, 치 아바타, 스프 등 고퀄의 메뉴를 즐길 수 있습니...
['신상', '이대', '브런치', '<', '포닐', '>', '정말', '맛있는', '브런치', '를', '즐길', '수', '있는', '이대', '브런치', '&', '이대', '맛집', '입니다', '.', '샌드위치', ',', '치', '아바타', ',', '스프', '등', '고', '퀄', '의', '메뉴', '를', '즐길', '수', '있', '습', '니', '...']
['신상', '이대', '브런치', '포닐', '정말', '브런치', '수', '이대', '브런치', '이대', '맛집', '샌드위치', '치', '아바타', '스프', '등', '퀄', '메뉴', '수', '습']
[('신상', 'Noun'), ('이대', 'Noun'), ('브런치', 'Noun'), ('<', 'Punctuation'), ('포닐', 'Noun'), ('>', 'Punctuation'), ('정말', 'Noun'), ('맛있는', 'Adjective'), ('브런치', 'Noun'), ('를', 'Josa'), ('즐길', 'Verb'), ('수', 'Noun'), ('있는', 'Adjective'), ('이대', 'Noun'), ('브런치', 'Noun'), ('&', 'Punctuation'), ('이대', 'Noun'), ('맛집', 'Noun'), ('입니다', 'Adjective'), ('.', 'Punctuation'), ('샌드위치', 'Noun'), (',', 'Punctuation'), ('치', 'Noun'), ('아바타', 'Noun'), (',', 'Punctuation'), ('스프', 'Noun'), ('등', 'Noun'), ('고', 'Modifier'), ('퀄', 'Noun'), ('의', 'Josa'), ('메뉴', 'Noun'), ('를', 'Josa'),

In [47]:
#token 중 일부 명사만 사용
pos_set = ['Noun']
# 'Noun', 'Verb', 'Alpha' 이중에서 품사 고르기

def selected_tokenizer(sent, pos = pos_set) :
    t = Okt()
    # t.pos()로 입력된 문장에 품사 태깅
    # (토근, 품사) 상 리스트 반환
    return [x[0] for x in t.pos(sent) if len(x[0]) > 1 and x[1] in pos]
# apply로 함수 적용
tst['tokens'] = tst['Review_text'].apply(selected_tokenizer)
tst

,Platform,Number,Store,Address,Addr,Review_score,Review_counts,ID,Date,Review_text,Time,tokens
0,naver,1,포닐,서울 마포구 대흥로 183,서울 마포구,NaN,177,minsuksoft,3.30.토,일요일 기분 내기 위한 + 맛있기 위한 브런치로 완벽! 스테이크 샌드위치는 불고기가...,2024-04-07 - 20:09:27,"[일요일, 기분, 내기, 브런치, 완벽, 스테이크, 샌드위치, 불고기, 진짜, 로스..."
1,naver,1,포닐,서울 마포구 대흥로 183,서울 마포구,NaN,177,kgcstyle,3.9.토,신상 이대 브런치 <포닐> 정말 맛있는 브런치를 즐길 수 있는 이대 브런치 & 이대...,2024-04-07 - 20:09:28,"[신상, 이대, 브런치, 포닐, 정말, 브런치, 이대, 브런치, 이대, 맛집, 샌드..."
2,naver,1,포닐,서울 마포구 대흥로 183,서울 마포구,NaN,177,양팡80,4.5.금,새우 샌드위치랑 아이스아메리카노 주문했습니다 커 피도 맛있고 샌드위치도 맛있어요! ...,2024-04-07 - 20:09:28,"[새우, 샌드위치, 아이스, 아메리카노, 주문, 샌드위치, 점심, 직원]"
3,naver,1,포닐,서울 마포구 대흥로 183,서울 마포구,NaN,177,uo****,4.6.토,항상 지나가면서 유심히 보던 빵집&브런치집인데 운 좋게 소금빵 나오는 시간에 방문했...,2024-04-07 - 20:09:28,"[항상, 유심, 빵집, 브런치, 소금, 시간, 방문, 바질, 터키, 샌드위치, 가격..."
4,naver,1,포닐,서울 마포구 대흥로 183,서울 마포구,NaN,177,OH옌,3.13.수,이대역 치아바타 샌드위치 맛집🥪 일정사이에 시간 이 붕떠서 시간도 떼우고 점심을 먹...,2024-04-07 - 20:09:28,"[이대역, 치아바타, 샌드위치, 맛집, 정사, 시간, 붕떠, 시간, 점심, 먹기, ..."
5,Kakao,1,만동제과,서울 서대문구 연희로 32 1층 (창천동),서울 서대문구,4.3,303,앙앙,2024.01.01.,나의 2024년 1월 1일 첫 먹방은 그간 너무너무너무 먹고 싶었던 ‘만동제과 마늘...,2024-04-05 - 10:35:09,"[먹방, 그간, 제과, 마늘, 바게트, 소원, 성취, 왕복, 직원, 바게트, 소스,..."
6,Kakao,1,만동제과,서울 서대문구 연희로 32 1층 (창천동),서울 서대문구,4.3,303,니엘,2024.01.01.,너무 맛있게 먹어서 사진도 못남겼어요 마늘바게뜨가 촉촉 부들하고 진한 마늘의 단맛이...,2024-04-05 - 10:34:50,"[사진, 마늘, 바게뜨, 촉촉, 마늘, 단맛, 인상]"
7,Kakao,1,만동제과,서울 서대문구 연희로 32 1층 (창천동),서울 서대문구,4.3,303,ㅇㅅㅇ,2024.01.03.,친절 하십니다 근데 막 꼭 먹어야한다 정도는 아닌.것 같아요,2024-04-05 - 10:34:32,"[친절, 정도]"
8,Kakao,1,만동제과,서울 서대문구 연희로 32 1층 (창천동),서울 서대문구,4.3,303,램든고지,2024.01.04.,마늘바게트에 묻힌 빵들 너무 많음 홍국이 깨찰빵 도라이임,2024-04-05 - 10:34:13,"[마늘, 바게트, 홍국, 깨찰빵, 이임]"
9,Kakao,1,만동제과,서울 서대문구 연희로 32 1층 (창천동),서울 서대문구,4.3,303,토끼토끼🐰,2024.01.04.,마늘빵 달달하고 촉촉하니 맛있어요. 카푸치노는 커피맛나는 생크림빵인데 구름처럼 슈...,2024-04-05 - 10:33:55,"[마늘빵, 달달, 카푸치노, 커피, 맛나, 생크림, 구름, 슈르륵, 우유, 환상, ..."


In [22]:
# 반복문 만들기
for i in range(10):
    rvw = 0
    rvw = tst['Review_text'][i]
    print(okt.nouns(rvw))  


['일요일', '기분', '내기', '위', '위', '브런치', '완벽', '스테이크', '샌드위치', '불고기', '진짜', '로스', '느', '낌', '고기', '식', '맛']
['신상', '이대', '브런치', '포닐', '정말', '브런치', '수', '이대', '브런치', '이대', '맛집', '샌드위치', '치', '아바타', '스프', '등', '퀄', '메뉴', '수', '습']
['새우', '샌드위치', '아이스', '아메리카노', '주문', '피', '샌드위치', '점심', '포', '분', '직원']
['항상', '유심', '빵집', '브런치', '집', '운', '소금', '빵', '시간', '방문', '바질', '터키', '햄', '샌드위치', '첨', '가격', '브레드']
['이대역', '치아바타', '샌드위치', '맛집', '정사', '시간', '이', '붕떠', '시간', '점심', '먹기', '위해', '방문', '곳', '역', '매장', '수']
['나', '첫', '먹방', '그간', '제과', '마늘', '바게트', '소원', '성취', '왕복', '직원', '바게트', '소스', '흠뻑', '혼자', '맛', '도리', '살짝', '못', '분', '입', '수']
['사진', '마늘', '바게뜨', '촉촉', '마늘', '단맛', '인상']
['친절', '막', '꼭', '정도', '것']
['마늘', '바게트', '빵', '홍국', '깨찰빵', '이임']
['마늘빵', '달달', '카푸치노', '커피', '맛나', '생크림', '빵', '구름', '슈르륵', '우유', '환상', '단맛', '어니언', '겉', '베이글', '사이', '크림', '치즈', '양파', '엉덩이', '빵', '빵안', '젤리', '우유', '크림', '원래', '빵', '우유', '젤리', '때문', '덜', '밀가루', '식감', '불호']


In [26]:
# 원하는 단어와 그 단어의 앞뒤 단어를 선택하는 함수
def select_context_words(sent, target_word, window=1, pos=['Noun']):
    t = Okt()
    tokens = t.pos(sent)
    selected_tokens = [x for x in tokens if x[1] in pos]
    
    # 결과를 저장할 리스트
    result = []
    
    # 선택된 토큰들을 순회하면서 조건에 맞는 경우에만 결과에 추가
    for i, token in enumerate(selected_tokens):
        if token[0] == target_word:
            # 현재 토큰의 앞뒤 window 크기만큼의 토큰들을 선택
            context = selected_tokens[max(i-window, 0):i+window+1]
            result.extend(context)
    
    # 중복 제거
    result = list(set(result))
    
    return result

# 예시 문장
# sent = "오늘 날씨가 참 좋네요. 내일도 좋았으면 좋겠어요."
for i in range(10):
    sent = 0
    sent = tst['Review_text'][i]

# 함수 실행: '좋네요'를 포함한 앞뒤 단어 선택
result = select_context_words(sent, ['빵','샌드위치','커피','베이글','바게트','바게뜨'], window=1)
print(result)

[]


In [29]:
# 원하는 단어와 그 단어의 앞뒤 단어를 선택하는 함수
def select_context_words(sent, target_words, window=1, pos=['Noun']):
    t = Okt()
    tokens = t.pos(sent)
    selected_tokens = [x for x in tokens if x[1] in pos]
    
    # 결과를 저장할 리스트
    result = []
    
    # 선택된 토큰들을 순회하면서 조건에 맞는 경우에만 결과에 추가
    for i, token in enumerate(selected_tokens):
        if token[0] in target_words:  # 리스트 내의 단어와 비교
            # 현재 토큰의 앞뒤 window 크기만큼의 토큰들을 선택
            context = selected_tokens[max(i-window, 0):i+window+1]
            result.extend(context)
    
    # 중복 제거
    result = list(set(result))
    
    return result

# 여기서 'your_dataframe'은 실제 데이터프레임 변수명으로 대체해야 합니다.
for i in range(10):  # 데이터프레임의 처음 10개 행에 대해 반복
    sent = tst['Review_text'][i]
    # 함수 실행: 지정한 단어들을 포함한 앞뒤 단어 선택
    result = select_context_words(sent, ['빵', '샌드위치', '커피', '베이글', '바게트', '바게뜨'], window=1)
    print(result)

[('샌드위치', 'Noun'), ('불고기', 'Noun'), ('스테이크', 'Noun')]
[('샌드위치', 'Noun'), ('맛집', 'Noun'), ('치', 'Noun')]
[('아이스', 'Noun'), ('점심', 'Noun'), ('샌드위치', 'Noun'), ('새우', 'Noun'), ('피', 'Noun')]
[('샌드위치', 'Noun'), ('햄', 'Noun'), ('첨', 'Noun'), ('빵', 'Noun'), ('시간', 'Noun'), ('소금', 'Noun')]
[('치아바타', 'Noun'), ('샌드위치', 'Noun'), ('맛집', 'Noun')]
[('소원', 'Noun'), ('소스', 'Noun'), ('마늘', 'Noun'), ('바게트', 'Noun'), ('직원', 'Noun')]
[('마늘', 'Noun'), ('바게뜨', 'Noun'), ('촉촉', 'Noun')]
[]
[('빵', 'Noun'), ('홍국', 'Noun'), ('마늘', 'Noun'), ('바게트', 'Noun')]
[('원래', 'Noun'), ('빵', 'Noun'), ('겉', 'Noun'), ('사이', 'Noun'), ('엉덩이', 'Noun'), ('우유', 'Noun'), ('카푸치노', 'Noun'), ('구름', 'Noun'), ('생크림', 'Noun'), ('빵안', 'Noun'), ('커피', 'Noun'), ('맛나', 'Noun'), ('베이글', 'Noun')]


In [33]:
# 원하는 단어와 그 단어의 앞뒤 단어를 선택하는 함수
def select_context_words(sent, target_words, window=1, pos=['Noun']):
    t = Okt()
    tokens = t.pos(sent)
    selected_tokens = [x for x in tokens if x[1] in pos]
    
    # 결과를 저장할 리스트
    result = []
    
    # 선택된 토큰들을 순회하면서 조건에 맞는 경우에만 결과에 추가
    for i, token in enumerate(selected_tokens):
        if token[0] in target_words:  # 리스트 내의 단어와 비교
            # 현재 토큰의 앞뒤 window 크기만큼의 토큰들을 선택
            context = selected_tokens[max(i-window, 0):i+window+1]
            result.extend(context)
    
    # 중복 제거
    result = list(set(result))
    
    # 결과 리스트에서 각 튜플의 첫 번째 요소(단어)만 추출
    first_elements = [x[0] for x in result]
    
    return first_elements

# 여기서 'your_dataframe'은 실제 데이터프레임 변수명으로 대체해야 합니다.
for i in range(10):  # 데이터프레임의 처음 10개 행에 대해 반복
    sent = tst['Review_text'][i]
    # 함수 실행: 지정한 단어들을 포함한 앞뒤 단어 선택 및 명사만 출력
    result = select_context_words(sent, ['빵', '샌드위치', '커피', '베이글', '바게트', '바게뜨', '맛집'], window=1)
    print(result)


['샌드위치', '불고기', '스테이크']
['샌드위치', '맛집', '치', '이대']
['아이스', '점심', '샌드위치', '새우', '피']
['샌드위치', '햄', '첨', '빵', '시간', '소금']
['치아바타', '샌드위치', '정사', '맛집']
['소원', '소스', '마늘', '바게트', '직원']
['마늘', '바게뜨', '촉촉']
[]
['빵', '홍국', '마늘', '바게트']
['원래', '빵', '겉', '사이', '엉덩이', '우유', '카푸치노', '구름', '생크림', '빵안', '커피', '맛나', '베이글']


# 합치기

In [6]:
# 인덱스 재설정
tst = tst.reset_index(drop=True)

pos_set = ['Noun']  # 사용할 품사 설정

# 특정 품사의 토큰만 선택하는 함수
def selected_tokenizer(sent, pos=pos_set):
    t = Okt()
    return [x[0] for x in t.pos(sent) if len(x[0]) > 1 and x[1] in pos]

# 토큰화 적용
# tst['tokens'] = tst['Review_text'].apply(selected_tokenizer)

# 원하는 단어와 그 단어의 앞뒤 단어를 선택하는 함수
def select_context_words(sent, target_words, window=1, pos=['Noun']):
    t = Okt()
    tokens = t.pos(sent)
    selected_tokens = [x for x in tokens if x[1] in pos]
    
    result = []
    for i, token in enumerate(selected_tokens):
        if token[0] in target_words:
            context = selected_tokens[max(i-window, 0):i+window+1]
            result.extend(context)
    result = list(set(result))  # 중복 제거
    # 결과 리스트에서 각 튜플의 첫 번째 요소(단어)만 추출
    first_elements = [x[0] for x in result]
    return first_elements

# 'Keyword' 칼럼 추가 및 결과 저장
# keyword = ['소금빵', '샌드위치', '케잌', '케익', '초콜릿', '케이크', '에스프레소', '크로와상', '마카롱', '타르트', '바게트', '치아바타', '머핀', '스콘', '파운드케이크', '파이', 
#            '브리오슈', '커스터드', '프레즐', '단팥빵','허니', '브레드', '감자', '티라미수', '베이글', '루스케이크', '무스', '롤케', '과일', '타르트', '크림빵', '치즈', '초코',
#            '블루베리', '초코칩', '쿠키', '마들렌', '휘낭시에', '팬케', '홍차', '페퍼민트', '카모마일', '레몬', '자몽', '오렌지', '토마토', '딸기','망고', '그린티', '얼그레이', '자스민', '맛집', '단팥', '빵', '식빵', '마카다미아', 
#            '루이보스', '페카노트', '베리', '코코넛', '버터크림', '크림' '아몬드', '크루아상','크루와상', '단호박', '고구마', '라즈베리', '블랙포레스트', '오페라케이크', '아이스티', '쌀', '디저트', '비건', '통밀', 
#            '복숭아', '호두', '피칸', '초콜릿', '바나나', '당근', '커피', '소라빵', '우유', '곡물', '허니글레이즈드', '메이플시럽', '버터스카치', '모카', '헤이즐넛', '글루텐', '땅콩', '사과', '꿀', '소보로', '찐득', '달달',
#            '카라멜', '멜론', '피스타치오', '피넛버터', '코코아', '녹차', '유자', '생강', '계피', '민트초코', '민트', '버터', '아이싱', '잼', '비치디다마','부드러운', '촉촉', '담백', '고소', '바삭', '신선', '호밀', '마스카포네',
#            '풍부', '달콤', '짭짤', '밀가루', '풍미', '발효',  '향', '겉바속촉', '사르르', '크럼블', '토핑', '씨앗','퓨레', '칩', '견과류', '말랑', '푹신', '쫀득', '커스터드', '키쉬','잠봉','퀸아망', '바닐라', '상큼', '에끌레어',
#            '건조', '실키', '층이 느껴지는', '에어리', '무거운', '가벼운', '크리미', '끈적', '부스러기', '수제', '아티산', '전통' ,'방식', '유기농', '재료', '시나몬', '밤', '무화과', '건포도', '살구', '프레즐', '프레첼', '크로플', '와플',
#            '바게트', '포카치아','그레인', '파니니', '치아바타','브라우니', '몽블랑', '바질', '말차']
keyword = ['소금빵', '샌드위치', '케잌', '케익', '초콜릿', '케이크','크로와상', '마카롱', '타르트', '바게트', '치아바타', '머핀', '스콘', '파운드케이크', '파이','브리오슈', '커스터드', '프레즐', 
           '단팥빵','허니', '브레드', '감자', '티라미수', '베이글', '루스케이크', '무스', '롤케', '과일', '타르트', '크림빵', '치즈', '초코','블루베리', '초코칩', '쿠키', '마들렌', '휘낭시에', '팬케', '홍차', '페퍼민트', '카모마일', 
           '레몬', '자몽', '오렌지', '토마토', '딸기','망고', '그린티', '얼그레이', '자스민', '맛집', '단팥', '빵', '식빵', '마카다미아','베리', '코코넛', '버터크림', '크림' '아몬드', '크루아상','크루와상', '단호박', '고구마', '라즈베리',
           '블랙포레스트', '오페라케이크', '아이스티', '쌀', '디저트', '비건', '통밀','복숭아', '호두', '피칸', '초콜릿', '바나나', '당근', '커피', '소라빵', '우유', '곡물', '허니글레이즈드', '메이플시럽', '버터스카치', '모카', '헤이즐넛', 
           '글루텐', '땅콩', '사과', '꿀', '소보로', '카라멜', '멜론', '피스타치오', '피넛버터', '코코아', '녹차', '유자', '생강', '계피', '민트초코', '민트', '버터', '아이싱', '잼', '비치디다마','밀가루', '발효', '크럼블', '퓨레', '칩',
           '토핑', '견과류', '커스터드', '에끌레어','키쉬','잠봉','퀸아망', '바닐라','유기농','시나몬', '밤', '무화과', '건포도', '살구', '프레즐', '프레첼', '크로플', '와플','바게트', '포카치아','그레인', '파니니', '치아바타','브라우니', 
           '몽블랑', '바질', '말차']
tst['Keyword'] = tst['Review_text'].apply(lambda sent: select_context_words(sent, keyword, window=1))
tst

,Number,Store,Address,Addr,Review_score,Review_counts,ID,Date,Review_text,Time,slope,Keyword
0,467,브라운홀릭,서울 강남구 강남대로98길 25,서울 강남구,4.44,918,여름이었ㄷr,23.5.19.금,후…훗…추 향과ㅜ맛이 마지막에 사알짝~,2024-04-08 - 03:17:27,6.09231,[]
1,381,바사카,서울 강남구 논현로85길 52 푸르지오시티1층 후문 바사카,서울 강남구,4.30,117,라미9132,23.9.19.화,맛잇어요 오리지날피자 / 배생강차는 저랑 좀 안 맞네요 / 다음엔 다른 피자 먹으러...,2024-04-08 - 03:09:29,0.75520,[]
2,1100,퍼스트커피랩홍대점,서울 마포구 신촌로4길 9 1층,서울 마포구,4.70,926,osh90,23.8.23.수,입시반 벱비들과 면접 대비 수업하러 카페 넘어옴 :)커피도 맛나고 빵도 맛남😋,2024-04-07 - 23:55:27,4.05963,"[커피, 카페, 맛남, 맛, 빵]"
3,233,런던베이글뮤지엄안국점,서울 종로구 북촌로4길 20 연화빌딩 1층 (계동),서울 종로구,3.00,606,cheese,2023.01.06.,궁금했었는데 지나가던 길에 방문했어요 웨이팅이 너~~~무 길어 전 포장했어요 포...,정보없음,0.00000,[]
4,104,꿀넹쿠키연남점,서울 마포구 동교로 255-1 1층,서울 마포구,4.76,971,Colorday,23.9.16.토,"글루텐프리라서 특별한 쿠키 맛집이에용 😄 우선 밀가루 자체가 좋고 달달한 쿠키, 달...",2024-04-07 - 21:49:32,1.96779,"[달달, 용, 우선, 자체, 구분, 쿠키, 약과, 맛집, 달, 라테, 프리, 글루텐..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,691,아오이토리,서울 마포구 와우산로29길 8 K.C빌딩,서울 마포구,4.44,3403,잇츠국화,23.12.21.목,"📋메론빵, 야키소바빵, 아메리카노빵은 꽤 괜찮은 편이였는데이 것만 먹으러 여기까지 ...",2024-04-07 - 20:41:17,1.90902,"[다소, 빵, 아메리카노, 꽤]"
996,277,리치몬드과자점성산본점,서울 마포구 월드컵북로 86,서울 마포구,4.48,4776,나니8635,3.10.일,모든 빵이 다 맛있어요! 리치몬드 믿고 먹습니다,2024-04-06 - 00:13:48,6.06511,"[리치몬드, 모든, 빵]"
997,376,밀토스트,서울특별시 종로구 수표로28길 30-3,서울 종로구,3.90,564,정보없음,정보없음,특제 찐 토스트의 본연의 맛과 밤맛이 모두 맛있습니다. 쪄서 빵 껍질이 너무 젖어 ...,2024-04-07 - 02:58:30,0.00000,"[정밀, 과열, 껍질, 모두, 드립, 커피, 빵]"
998,1002,코소베이커리,서울 성북구 돌곶이로 220 상가동 1층 105-102,서울 성북구,4.40,112,jju****,23.3.26.일,저희가족 요기 빵 좋아해서매일가고있어요^^,2024-04-07 - 23:41:34,0.33768,"[매일, 빵, 요기]"


In [7]:
tst[159:174]

,Number,Store,Address,Addr,Review_score,Review_counts,ID,Date,Review_text,Time,slope,Keyword
159,1058,테이퍼드커피,"서울 강남구 선릉로 622 지상1층(삼성동, 대영빌딩)",서울 강남구,4.40,598,짼블리,23.9.7.목,테이퍼드커피 옥수점 자주 가는데 선정릉에도 있어서 가봤어요! 커피 맛있고 인테리어도...,2024-04-08 - 04:02:10,3.93095,"[커피, 퍼드, 인테리어, 옥수, 선정릉]"
160,127,뇽뇽마카롱,서울 마포구 희우정로 72 1층 뇽뇽마카롱,서울 마포구,4.40,279,beloved2456,23.1.3.화,자주가는 맛있는 곳입니다,2024-04-06 - 01:53:22,0.95528,[]
161,1033,크림데이글루텐프리케이크샵,서울 마포구 월드컵로21길 39,서울 마포구,4.80,1240,건강소녀냥,3.17.일,쿠키가 새로나와서 먹으러 왔어요~다른곳보다 쿠키가 커서 좀더 비싸요.,2024-04-07 - 23:22:20,1.35049,"[쿠키, 커서, 곳]"
162,1221,호랑가시,서울 강남구 봉은사로6길 32 지하1층-3층,서울 강남구,3.90,994,빵미나링,2.29.목,커피는 맛있는데 디저트가 좀 비쌉니다.호랑슈페너는... 크림이 너무 적어서 아쉬웠어...,2024-04-08 - 01:36:23,5.39729,"[디저트, 좀, 커피]"
163,808,오월의밀밭,서울 성북구 종암로19길 40 1층,서울 성북구,4.54,323,yuns4,23.4.17.월,일찍가야해요 소금빵사러~,2024-04-08 - 02:12:12,2.13438,"[빵, 소금]"
164,954,카페라미스콘청담점,서울 강남구 도산대로53길 32,서울 강남구,4.40,1044,riramii,23.9.28.목,달지 않아서 아이와 부담없이 잘 먹었습니다:),2024-04-08 - 02:09:32,1.01295,[]
165,744,애니오케이션카페,서울 강남구 도산대로51길 14 1층,서울 강남구,4.30,791,둡둘,22.10.21.금,"와우 연어베이글도 맛이지만(연어 너무 신선), 토스팅 된 베이글이 너무너무 맛있어요...",2024-04-08 - 03:08:47,1.72117,"[조합, 소금, 넛츠, 베이글, 크림, 스팅, 치즈, 맛, 연어, 카라멜]"
166,932,치즈인더스트리[치즈공업사],서울 종로구 수표로28길 33-7,서울 종로구,4.30,659,델푸트 dailyfoodtrip81,23.10.29.일,글로우 서울이 만든 핸드메이드 치즈와 다양한 변주를 만날 수 있는 치즈 그로서리 마...,2024-04-06 - 17:58:25,0.95478,"[수, 카페, 수제, 빵, 변주, 폼, 바깥, 직접, 커피, 과정, 인더스, 케이크..."
167,322,멜로디케이크,서울 마포구 백범로 13 4층 408호,서울 마포구,4.40,118,kdamh714,22.10.24.월,인스타 뒤져보다가 디자인이 너무 마음에 들어서 연락드렸는데 사장님이 정말 친절하세요...,2024-04-06 - 01:51:55,3.19972,"[티라미수, 맛, 저, 케이크]"
168,691,아오이토리,서울 마포구 와우산로29길 8 KC빌딩 1층 (서교동),서울 마포구,4.00,234,워킹둘기,2023.09.16.,미치게 맛있습니다.. 메론빵도 정말 맛있습니다. 이런 빵들 좋아하시면 굳이와서 드실...,2024-04-07 - 02:42:06,1.90902,"[정말, 빵, 굳이]"
